In [1]:
!pip install stanza
!pip install textblob
!pip install xgboost


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import time
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
import random
import stanza
import re

In [3]:
import pandas as pd
import numpy as np
import re
import warnings
import torch
from sentence_transformers import SentenceTransformer
import emoji
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import nltk

In [4]:
import sklearn
sklearn.__version__
from sklearn.utils import class_weight
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import decomposition, ensemble
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn import neighbors
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_validate
from sklearn.linear_model import SGDClassifier

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.utils import np_utils

In [6]:
import itertools
from textblob import TextBlob 
import seaborn as sns
import xgboost as xgb
from xgboost import XGBClassifier
sns.set(style="darkgrid")

In [7]:
random.seed(42)

In [8]:
import string
from tqdm import tqdm
# ---- Call tqdm to see progress bar with pandas
tqdm().pandas()

0it [00:00, ?it/s]


In [9]:
class PreProcessing(object):
    '''
    Class to preprocess text
    
    '''
    
    
    def __init__(self):
        #print("Welcome in the preprocessing")
        pass
    
    @classmethod    
    def detect_lang_google(self, x):
        '''
        Function to detect the language of the string
        @param x: (str) sentences of text to detect language
        @return: (str or nan) language of the sentence
        '''
        translate = Translator()
        try:
            return translate.detect(x).lang
        except:
            return np.nan
    
    @classmethod
    def remove_numbers(self, text):
        '''
        Function to remove number in text.
        @param text: (str) sentence
        @return: (str) clean text
        '''
        text = ''.join([i for i in text if not i.isdigit()])         
        return text
    
    @classmethod
    def remove_URL(self, text):
        '''
        Function to remove url from text.
        @param text: (str) sentence
        @return: (str) clean text
        
        '''
        url = re.compile(r'https?://\S+|www\.\S+')
        return url.sub(r'',text)
    
    @classmethod
    def remove_html(self, text):
        '''
        Function regex to clean text from html balises.
        @param text: (str) sentence 
        @return: (str) clean text 
        '''
        html=re.compile(r'<.*?>')
        return html.sub(r'',text)
    
    
    @classmethod
    def remove_emoji(self, text):
        '''
        Function to remove emojis, symbols and pictograms etc from text
        @param text: (str) sentences 
        @return: (str) clean text 
        '''
        emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)
    
    
    @classmethod
    def preprocess(self, text):
        '''
        Function to remove special characters
        @param text: (pandas dataframe) text
        @return: (pandas dataframe) clean text 
        '''
        text = text.replace("(<br/>)", "")
        text = text.replace('(<a).*(>).*(</a>)', '')
        text = text.replace('(&amp)', '')
        text = text.replace('(&gt)', '')
        text = text.replace('(&lt)', '')
        text = text.replace('(\xa0)', ' ')  
        text = text.replace("\n", " ")
        text = text.replace("\x92", "'")
        return text
    
    @classmethod
    def remove_char_specific(self, text):
        '''
        Function to remove specific characters
        @param text: (str) text
        @return: (str) text without specific characters
        '''
        table = '!"#$%&()*+,./:;<=>?@[\]^_`{|}~•'
        table = str.maketrans(' ', ' ', table)
        words = text.split()
        stripped = [w.translate(table) for w in words]
        return ' '.join(stripped)
    
    @classmethod
    def remove_upper_case(self, text):
        '''
        Function to transform upper string in title words
        @param text: (str) text 
        @return: (str) text without upper words 
        '''
        words = text.split()
        stripped = [w.title() if w.isupper() else w for w in words]
        return " ".join(stripped)
    
    @classmethod
    def remove_stop_words(self, x, stop_word):
        '''
        Function to remove a list of words
        @param x : (str) text 
        @param stop_word: (list) list of stopwords to delete 
        @return: (str) new string without stopwords 
        '''
        x_new = text_to_word_sequence(x)    # tokenize text 
        x_ = []
        for i in x_new:
            if i not in stop_word:
                x_.append(i)
        return " ".join(x_)
    
    @classmethod
    def get_top_n_words(self, corpus, n=None):
        '''
        Function to return a list of most frequent unigrams in documents
        @param corpus: (str or pandas.dataframe) documents 
        @param n: (int) number of most frequent unigrams
        @return: (list) most frequent unigrams
        '''
        vec = CountVectorizer().fit(corpus)             # bag of words
        bag_of_words = vec.transform(corpus)
        sum_words = bag_of_words.sum(axis=0)  
        words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        return words_freq[:n]
    
    @classmethod
    def get_top_n_words_sw(self, corpus, stop_word=None, lang="fr", n=None):
        '''
        Function to return a list of most frequent unigrams in documents
        @param corpus: (str or pandas.dataframe) documents 
        @param stop_word: (list) list containing stopwords
        @param lang: (str) language of the text
        @param n: (int) number of most frequent unigrams
        @return: (list) most frequent unigrams
        '''
        if lang=="fr":
            corpus = corpus.apply(lambda x: self.remove_stop_words(x, stop_word))
        vec = CountVectorizer(stop_words = "english").fit(corpus)
        bag_of_words = vec.transform(corpus)
        sum_words = bag_of_words.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        return words_freq[:n]
    
    @classmethod
    def get_top_n_bigram(self, corpus, n=None):
        '''
        Function to return a list of most frequent bigrams in documents
        @param corpus: (str or pandas.dataframe) documents 
        @param n: (int) number of most frequent unigrams
        @return: (list) most frequent unigrams
        '''
        vec = CountVectorizer(ngram_range=(2, 2)).fit(corpus) 
        bag_of_words = vec.transform(corpus)
        sum_words = bag_of_words.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        return words_freq[:n]
    
    @classmethod
    def get_top_n_bigram_sw(self, corpus, stop_word=None, lang="fr", n=None):
        '''
        Function to return a list of most frequent bigrams in documents
        @param corpus: (str or pandas.dataframe) documents 
        @param stop_word: (list) list containing stopwords
        @param lang: (str) language of the text
        @param n: (int) number of most frequent unigrams
        @return: (list) most frequent unigrams
        '''
        if lang=="fr":
            corpus = corpus.apply(lambda x: self.remove_stop_words(x, stop_word))
        vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
        bag_of_words = vec.transform(corpus)
        sum_words = bag_of_words.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        return words_freq[:n]
    
    @classmethod
    def get_top_n_trigram(self, corpus, n=None):
        '''
        Function to return a list of most frequent trigrams in documents
        @param corpus: (str or pandas.dataframe) documents 
        @param n: (int) number of most frequent unigrams
        @return: (list) most frequent unigrams
        '''
        vec = CountVectorizer(ngram_range=(3, 3)).fit(corpus)
        bag_of_words = vec.transform(corpus)
        sum_words = bag_of_words.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        return words_freq[:n]

    @classmethod
    def get_top_n_trigram_sw(self, corpus, stop_word=None, lang="fr", n=None):
        '''
        Function to return a list of most frequent trigrams in documents
        @param corpus: (str or pandas.dataframe) documents 
        @param stop_word: (list) list containing stopwords
        @param lang: (str) language of the text
        @param n: (int) number of most frequent unigrams
        @return: (list) most frequent unigrams
        '''
        if lang=="fr":
            corpus = corpus.apply(lambda x: self.remove_stop_words(x, stop_word))
        vec = CountVectorizer(ngram_range=(3, 3), stop_words="english").fit(corpus)
        bag_of_words = vec.transform(corpus)
        sum_words = bag_of_words.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        return words_freq[:n]
    
    @classmethod
    def get_top_n_5grams_sw(self, corpus, stop_word=None, lang="fr", n=None):
        '''
        Function to return a list of most frequent trigrams in documents
        @param corpus: (str or pandas.dataframe) documents 
        @param stop_word: (list) list containing stopwords
        @param lang: (str) language of the text
        @param n: (int) number of most frequent unigrams
        @return: (list) most frequent unigrams
        '''
        if lang=="fr":
            corpus = corpus.apply(lambda x: self.remove_stop_words(x, stop_word))
        vec = CountVectorizer(ngram_range=(5, 5), stop_words="english").fit(corpus)
        bag_of_words = vec.transform(corpus)
        sum_words = bag_of_words.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
        words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
        return words_freq[:n]

In [10]:
class Metrics(object):
    '''
    Class containing functions to plot the different metric curves (Precision-Recall, ROC AUC etc...)
    '''
    
    def __init__(self):
        '''
        Initialisation of the class'''
        
        

    @classmethod 
    def roc_auc_curve(self, model, x, y, labels, gb=False):
        '''
        Function to plot the ROC AUC curves for binary or multiclass classification. 
        Correct for standard machine learning models and Neural Networks. 
        @param model: (model) classification model
        @param x: (list) validation sample
        @param y: (list int) validation sample label
        @param gb: (bool) inform if the model is an ensemble model 
        '''
        # generate a no skill prediction (majority class)
        ns_probs = [0 for _ in range(len(y.reshape(-1, 1)))]
        # predict probabilities
        if gb: # test if ensemble model 
            lr_probs = model.predict_proba(x)
        else:    
            lr_probs = model.predict(x)
        
        plt.figure(figsize=(10,8))
        if len(labels)==2: # binary classification 
            if gb:
                # compute area under the roc curve
                lr_auc = roc_auc_score(y, lr_probs[:,1], average="weighted")
            else:
                lr_auc = roc_auc_score(y, lr_probs, average="weighted")
            # Compute no skill roc curve 
            ns_fpr, ns_tpr, _ = roc_curve(y, ns_probs)
            # plot the curve no skill
            plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
            if gb: # test if the model is an ensemble model
                # compute ROC curve
                lr_fpr, lr_tpr, _ = roc_curve(y, lr_probs[:,1])
            else:
                # compute ROC curve
                lr_fpr, lr_tpr, _ = roc_curve(y, lr_probs)

            # plot the roc curve for the model
            plt.plot(lr_fpr, lr_tpr, label=f'Class (area {round(lr_auc,3)})')
                # axis labels
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
                # show the grid
            plt.grid(True)
                # show the legend
            plt.legend()
        else: # multilabel classification 
            dummy_y = np_utils.to_categorical(y)
            #ns_auc = roc_auc_score(valid_y, dummy_ns, average="weighted", multi_class="ovr")

            lr_auc_multi = []
            for i in enumerate(labels):
                lr_auc_multi.append(round(roc_auc_score(dummy_y[:,i[0]], lr_probs[:,i[0]], average="weighted"),3))
                print(f"ROC AUC class {i[1]}: {lr_auc_multi[-1]}")
            lr_auc = roc_auc_score(dummy_y, lr_probs, average="weighted", multi_class="ovr" )

            ns_fpr, ns_tpr = [i/10 for i in list(range(0, 11, 1))], [i/10 for i in list(range(0, 11, 1))] #f(range(0, 0.1, 1))
            plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
            for i in range(lr_probs.shape[1]):
                lr_fpr, lr_tpr, _ = roc_curve(dummy_y[:,i], lr_probs[:,i])
                # plot the roc curve for the model
                plt.plot(lr_fpr, lr_tpr, label=f'Class {labels[i]} (area {lr_auc_multi[i]})')
                # axis labels
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                # show the grid
                plt.grid(True)
                # show the legend
                plt.legend()
                
        print('\nROC AUC=%.3f \n' % (lr_auc))
        plt.show()

    @classmethod 
    def confusion_matrix(self, model, y, x, labels):
        '''
        Compute the confusion matrix for binary or multiclass classification. 
        Correct for standard machine learning models and Neural Networks. 
        @param model: (model) classification model
        @param x: (list) validation sample
        @param y: (list int) validation sample label
        
        '''
        if len(labels)==2: # binary confusion matrix
            confu_matrix = pd.DataFrame(confusion_matrix(y, (model.predict(x) > 0.5).astype(int)), \
                 columns=['Predicted Negative', "Predicted Positive"], index=['Actual Negative', 'Actual Positive'])
            print(confu_matrix)
            return confu_matrix
        else:
            # multiclass confusion matrix
            dummy_y = np_utils.to_categorical(y)
            mcm = multilabel_confusion_matrix(dummy_y, np_utils.to_categorical(model.predict(x).argmax(-1)))
            df_mcm = pd.DataFrame()
            for i in zip(mcm, labels): # compute confusion matrix for each class 
                mcm = pd.DataFrame(data=i[0], columns=['Predicted Negative', "Predicted Positive"], index=['Actual Negative', 'Actual Positive'])
                df_mcm = df_mcm.append(mcm)
                print("\nConfusion matrix for classe: %s \n" %(i[1]))
                print(mcm)
                print("\n")
            return df_mcm
        
    @classmethod 
    def precision_recall_curve(self, model, x, y, labels, gb=False):
        '''
        Function to plot the recall precision curves for binary or multiclass classification. 
        Correct for standard machine learning models and Neural Networks. 
        @param model: (model) classification model
        @param x: (list) validation sample
        @param y: (list int) validation sample label
        @param gb: (bool) inform if the model is an ensemble model 
        '''
        # predict probabilities
        if gb: # test if the model is an ensemble model 
            lr_probs = model.predict_proba(x)
        else:
            lr_probs = model.predict(x)

        print("\n")
        plt.figure(figsize=(10,8))

        if len(labels)==2: # binary classification 
            if gb:
                precision, recall, thresholds = precision_recall_curve(y, lr_probs[:,1]) # compute precision recall curve 
                lr_f1 = f1_score(y,(lr_probs[:,1]>0.5).astype(int))
            else:
                precision, recall, thresholds = precision_recall_curve(y, lr_probs)
                lr_f1 = f1_score(y,(lr_probs>0.5).astype(int))
            
            # calculate precision-recall AUC
            lr_auc = auc(recall, precision)
            # summarize scores
            print('Model: f1-score=%.3f AUC=%.3f' % (lr_f1, lr_auc)) # print f1-score and auc 
            plt.plot(recall, precision, marker='.', label='Model')
            no_skill = len(y[y==1]) / len(y)
            plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
        else:
            dummy_y = np_utils.to_categorical(y)
            dummy_lr = np_utils.to_categorical(lr_probs.argmax(-1))
            for i in enumerate(labels):
                precision, recall, thresholds = precision_recall_curve(dummy_y[:,i[0]], lr_probs[:,i[0]])
                # calculate precision-recall AUC
                lr_f1 = f1_score(dummy_y[:,i[0]], dummy_lr[:,i[0]]) 
                lr_auc = auc(recall, precision)
                # summarize scores
                print('Model class: %s --> f1-score=%.3f AUC=%.3f' % (i[1], lr_f1, lr_auc))
                plt.plot(recall, precision, label='Class %s' %(i[1]))
            no_skill = len(y[y>=1]) / len(y)
            plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
        # plot the precision-recall curves
        print("\n")

        # axis labels
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        # show the legend
        plt.legend()
        # show the grid
        plt.grid(True)
        # show the plot
        plt.show()

    @classmethod 
    def plot_eval_xgb(self, model, labels):
        '''
        Function to plot the evaluation curves for xgboost models 
        @param model: (model) xgboost model
        @param labels: (list) list ocntaining the labels in string 
        '''
        # retrieve performance metrics
        results = model.evals_result()
        if len(labels)>2: # multiclass 
            log_ = "mlogloss"
            error_= "merror"
        else: # binary classifiation
            log_ = "logloss"
            error_= "error"

        # create axis x with the number of epochs
        epochs = len(results['validation_0'][error_])
        x_axis = range(0, epochs)

        plt.figure(figsize=(15,10))
        plt.subplot(221)
        # Plot training & validation accuracy values
        plt.plot(x_axis, results['validation_0'][log_], label='Train')
        plt.plot(x_axis, results['validation_1'][log_], label='Test')
        plt.ylabel('Log Loss')
        plt.xlabel('Epochs')
        plt.title('XGBoost Log Loss')
        plt.legend(loc='upper left')
        plt.grid(True)


        # Plot training & validation loss values
        plt.subplot(222)
        plt.plot(x_axis, results['validation_0'][error_], label='Train')
        plt.plot(x_axis, results['validation_1'][error_], label='Test')
        plt.legend()
        plt.ylabel('Classification Error')
        plt.xlabel('Epochs')
        plt.title('XGBoost Classification Error')
        plt.legend( loc='upper left')
        plt.grid(True)
        plt.show()
        
    @classmethod 
    def plot_confusion_matrix(self, cm, classes, normalized=True, cmap='bone'):
        '''
        Function to generate an heatmap of the confusion matrix
        @param cm: (matrix) confusion matrix
        @param classes: (list) list containing labels of the classes
        @param normalised: (bool) determined if the confusion matrix need to be normalized
        @param cmap: (str) color for the confusion matrix
        '''
        plt.figure(figsize=[10, 8])
        norm_cm = cm
        if normalized:
            norm_cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            sns.heatmap(norm_cm, annot=cm, fmt='g', xticklabels=classes, yticklabels=classes, cmap=cmap)
            #plt.savefig('confusion-matrix.png')

    @classmethod
    def plot_history(self, history):
        '''
        Function to plot the learning curves of a neural network
        @param history: metrics of a neural network
        '''
        plt.figure(figsize=(15,10))
        plt.subplot(221)
        # Plot training & validation accuracy values
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Test'], loc='upper left')
        plt.grid(True)


        # Plot training & validation loss values
        plt.subplot(222)
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Test'], loc='upper left')
        plt.grid(True)
        plt.show()
            
            
    @classmethod
    def metrics_deep_learning(self, model, history, x, y, labels):
        '''
        Function to plot the different metrics for the deep learning algorithms.
        @param model: (tensorflow.python.keras.engine.sequential.Sequential) deep learning model
        @param history: (tensorflow.python.keras.callbacks.History) history of the training model
        @param x: (numpy.ndarray) x data
        @param y: (numpy.ndarray) target data
        @param labels: (list) list containing the labels in str  
        '''
        self.plot_history(history)
        if len(labels)==2:

            print(classification_report(y, (model.predict(x) > 0.5).astype(int), target_names=labels))
            print(f"\nThe balanced accuracy is : {round(100*balanced_accuracy_score(y, (model.predict(x)>0.5).astype(int)),2)}%\n")
            print(f"\nThe Zero-one Loss is : {round(100*zero_one_loss(y, (model.predict(x)>0.5).astype(int)),2)}%\n")
            print(f"\nExplained variance score: {round(explained_variance_score(y, (model.predict(x)>0.5).astype(int)),3)}\n" )
            self.roc_auc_curve(model, x, y, labels)
            self.precision_recall_curve(model, x, y, labels)

            print(f"\nCohen's kappa: {round(100*cohen_kappa_score(y, (model.predict(x) > 0.5).astype(int) ),2)}% \n") 
            #matrices = self.confusion_matrix(model, y, x, labels)
            cm = confusion_matrix(y, (model.predict(x) > 0.5).astype(int))

            print("\nConfusion Matrix\n")
            self.plot_confusion_matrix(cm, labels)
        else:

            print(f"\nThe balanced accuracy is : {round(100*balanced_accuracy_score(y, model.predict(x).argmax(axis=-1)),2)}%\n")
            print(f"\nThe Zero-one Loss is : {round(100*zero_one_loss(y, model.predict(x).argmax(axis=-1)),2)}%\n")
            print(f"\nExplained variance score: {round(explained_variance_score(y, model.predict(x).argmax(axis=-1)),3)}\n" ) 
            self.roc_auc_curve(model, x, y, labels)
            self.precision_recall_curve(model, x, y, labels)

            print(f"\nCohen's kappa: {round(100*cohen_kappa_score(y, model.predict(x).argmax(axis=-1) ),2)}%\n")
            #matrices = self.confusion_matrix(model, y, x, labels)
            cm = confusion_matrix(y, model.predict(x).argmax(axis=-1))

            print(classification_report(y, model.predict(x).argmax(axis=-1), target_names=labels))

            print("\nConfusion Matrix\n")
            self.plot_confusion_matrix(cm, labels)

In [11]:
def classifier_model(clf, X, y, x_test, y_test):
    '''
    Function to compute a classifier model
    @param clf: (model) classifier model
    @param X: (matrix) training x data
    @param y: (list) train labels - ground truth
    @param x_test: (matrix) matrix of test x data
    @param y_test: (list) list of test labels
    @return model_NB: (model) train model
    @return pred: (list) list of predicted labels
    @return end: (float) fit time of the model
    @return model.score(): (float) accuracy of the model
    '''
    
    start = time.time()
    clf.fit(X, y)
    end = time.time() - start
    pred = clf.predict(x_test)
    
    #accuracy = accuracy_score(y_test, pred)
    
    return clf, pred, end, clf.score(x_test, y_test)#, accuracy

In [12]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import zero_one_loss
from sklearn.metrics import explained_variance_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
def metrics_ML(clf, X, Y, X_test, Y_test, labels, gb=False):
    '''
    Function to compute a classifier model
    @param X: (matrix) training x data
    @param y: (list) train labels - ground truth
    @param x_test: (matrix) matrix of test x data
    @param y_test: (list) list of test labels
    @return model_NB: (model) train model
    @return pred: (list) list of predicted labels
    @return end: (float) fit time of the model
    @return model.score(): (float) accuracy of the model
    '''
    model_, pred, time_train, score_ = classifier_model(clf, X, Y, X_test, Y_test)
    print("Execution time : %.3f s" %(time_train))
    print(f"Score : {round(100*score_,2)} %" )
    print("\nClassification Report\n")
    print(classification_report(Y_test, pred, target_names=labels))
    cm = confusion_matrix(Y_test, pred)
    print("\nConfusion Matrix\n")
    Metric.plot_confusion_matrix(cm, labels)
    '''
    
    print("\n")
    Metric.precision_recall_curve(model_,  X_test, Y_test, labels, gb=gb)
    print("\n")
    Metric.roc_auc_curve(model_,  X_test, Y_test, labels, gb=gb)
    print(f"\n\nCohen's kappa: {round(100*cohen_kappa_score(Y_test,  pred),2)}%\n\n")
    '''
    

In [13]:
preproc = PreProcessing()
Metric = Metrics()

Loading the dataset

In [14]:
def load_text(path, n, label):
  x = []
  for i in range(n):
    T = ""
    #print(path)
    with open(str(path + str(i+1)+'.txt'),encoding="utf8") as f:
      lines = f.readlines()
      for e in lines:
        T = T + e
    x.append(T)
  L = np.zeros(len(x))
  if(label == 1):
    L = np.ones(len(x))
  return x, L

In [15]:
p = ['D:/Co/Sem7/Prj/MiniProj/Abusive/Abusive-',
     'D:/Co/Sem7/Prj/MiniProj/NonAbusive/Non-Abusive-']
N = [40, 47]
L = [0, 1]
Text_nab, Lbl_nab = load_text(p[1] ,N[0], L[0])
Text_ab, Lbl_ab = load_text(p[0], N[1], L[1])

In [17]:
Text_ab

['அஞ்சு வருஷம் உங்க பவர் காமிச்சீங்களே என்ன ஆட்டம் போட்டீங்க எடிமகே  சுண்ணிக்கே எங்க தளபதி அத தடுக்கத்தாண்டா வராரு அவுர மட்டும் அழிக்கணும்னு நினைச்ச முடியவே முடியாது நீ அதுக்கு தகுதியே கிடையாது எடிமகே ஒ சுண்ணிக்கே ஒ நீ அதுக்கு தகுதியே கிடையாது என் தளபதி தான் இந்த நாட்டையே இந்த நாட்டையே நல்லவிதமா மாத்துறது எங்க தளபதி மட்டும்தான் நீ அந்த அஞ்சு வருஷம் என்ன பண்ணமுடியும் இந்த மக்களுக்காக இந்த மக்கள் என்ன பண்ணப்போறாய்ங்க அந்த தைரியத்தில தானே நீக்க இப்படி ஆடுறிங்க இந்த சர்க்கார் படம் வந்ததுல தாண்ட தெரியுது உங்க ஊழல் புண்ட என்னடா புண்டைகளா சிரிப்புத்தாண்டா வருது உங்கள பாத்த உங்களப்பத்தி பேசுனாலே சிரிப்பு புண்ட தாண்ட வருது கோவா புண்டைய வருது ஜோக்கர் புண்டடா நீங்கல்லாம் இப்படி பேசுறாங்களே அவமான புண்டையா இல்லையா     ',
 'நரேந்திர மோடியா நீதிபதியா வந்து நரேந்திர மோடிக்கு மூனு ஆயுள் தண்டனை ரெண்டு தூக்கு தண்டனை போடுறமாரி நீதிபதியா வந்து நடி குஜராத் கலவரத்தை விசாரிக்கும் அதிகாரியவ நீதிதுறை அதிகாரியவ நடி சுப்ரீம் கோர்ட் ஜுட்ஜ் ஆய்க்கோ அதுல வேணும்ன்னா நானே வக்கீலா நடிக்குறேன் பரவலா என்னை அணைபோட்டு தூக்

In [18]:
Lbl_ab

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
Text = Text_nab + Text_ab
Lbl = np.concatenate((Lbl_nab, Lbl_ab), axis = 0)
len(Text), len(Lbl)

In [ ]:
df = pd.DataFrame(data=[Text, Lbl], index=["text", "label"]).T

In [ ]:
df.shape

In [ ]:
df["text"] = df["text"].progress_apply(preproc.remove_URL)
df["text"] = df["text"].progress_apply(preproc.remove_html)
df["text"] = df["text"].progress_apply(preproc.remove_emoji)

print(df.head())

In [ ]:
df.to_csv('h1.csv') 

In [ ]:
df['polarity'] = df["text"].map(lambda text: TextBlob(text).sentiment.polarity)

In [ ]:
# ---- Show the polarities repartition
df['polarity'].hist()
plt.xlabel("Polarity")
plt.ylabel("Number of text")
#plt.savefig("polarity.png")

In [ ]:
# ---- Extract statistics of the text 
df['char_count'] = df["text"].apply(len) # Number of characters in the string
df['word_count'] = df["text"].apply(lambda x: len(x.split())) # Number of words in the string 
df['word_density'] = df['char_count'] / (df['word_count']+1) # Density of word (in char)
df['punctuation_count'] = df["text"].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
df['title_word_count'] = df["text"].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()])) # Number of words containing upper letter in first place 
df['upper_case_word_count'] = df["text"].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [ ]:
# ---- Show the number of characters per document
plt.figure(figsize=(15,10))
max_x = 10000 if df.char_count.max()>10000 else df.char_count.max()
plt.hist(df.char_count.values, bins = range(0, max_x, 100))
plt.title(f"Number of Video Transcripts {df.shape[0]}")
plt.xlabel("Number of characters")
plt.ylabel("Number of documents")
plt.grid(True)
plt.show()
plt.savefig("numb_char.png")

In [ ]:
# ---- histogram of the class frequency
df["label"].hist(xrot=45)
plt.xlabel("Classes")
plt.ylabel("Frequency")
plt.title("Class Frequency")
plt.savefig("distrib_classes.png")

In [ ]:
from keras.preprocessing.text import text_to_word_sequence
df.loc[:,"text_sw"] = df.loc[:,"text"].progress_apply(lambda x : preproc.remove_stop_words(x, "/"))

In [ ]:
if df["text_sw"].isnull().sum()>0:
    print("Empty text")
    df["text_sw"][df["text_sw"].isnull()] = "empty_text"

In [ ]:
LBL = ["Non_Abusive", "Abusive"]

Loading the transformers 1


In [ ]:

model_1 = SentenceTransformer('sentence-transformers/LaBSE')
embeddings_1 = model_1.encode(np.array(df["text"]))

train_x_1, valid_x_1, y_train_1, y_valid_1 = model_selection.train_test_split(np.array(embeddings_1), df["label"], random_state=42, stratify=df["label"], test_size=0.2)

In [ ]:
y_train_1 = y_train_1.astype('int')
y_valid_1 = y_valid_1.astype('int')

In [ ]:
# SVM
svm_clf = SVC(kernel="poly",C=100,gamma=1)
svm_clf.fit(train_x_1,np.array(y_train_1))
svm_pred = svm_clf.predict(valid_x_1)
#print(classification_report(y_valid_1,svm_pred))

metrics_ML(svm.SVC(kernel="poly",C=100,gamma=1),  train_x_1, y_train_1, valid_x_1, y_valid_1, LBL, gb=False)


In [ ]:
# Random Forest
rf = ensemble.RandomForestClassifier()
rf.fit(train_x_1,y_train_1)
rf_pred = rf.predict(valid_x_1)
# print(classification_report(y_valid_1,rf_pred))

metrics_ML(ensemble.RandomForestClassifier(),  train_x_1, y_train_1, valid_x_1, y_valid_1, LBL, gb=False)

In [ ]:
# Logestic Regression
lr = linear_model.LogisticRegression(max_iter=1000,  random_state=42)
lr.fit(train_x_1,y_train_1)
lr_pred = lr.predict(valid_x_1)
# print(classification_report(y_valid_1,rf_pred))

metrics_ML(linear_model.LogisticRegression(max_iter=1000,  random_state=42),  train_x_1, y_train_1, valid_x_1, y_valid_1, LBL, gb=True)

In [ ]:
# SGD
print("\nStochastic Gradient Descent with early stopping for One-Hot encoding\n")
print("Early Stopping : 10 iterations without change")
sgd = SGDClassifier(loss='modified_huber', max_iter=1000, tol=1e-3,   n_iter_no_change=10, early_stopping=True, n_jobs=-1 )
sgd.fit(train_x_1,y_train_1)
sgd_pred = sgd.predict(valid_x_1)
#print(classification_report(y_valid_1,sgd_pred))

metrics_ML(SGDClassifier(loss='modified_huber', max_iter=1000, tol=1e-3,   n_iter_no_change=10, early_stopping=True, n_jobs=-1 ),  train_x_1, y_train_1, valid_x_1, y_valid_1, LBL, gb=True)

In [ ]:
# XgBOOST
xbg = XGBClassifier(n_estimators=1000, subsample=0.8)
eval_set = [(train_x_1, y_train_1), (valid_x_1, y_valid_1)]
xbg.fit(train_x_1, y_train_1, eval_metric=["error", "logloss"],  eval_set=eval_set,early_stopping_rounds=10, verbose=False) 
xgb_pred = xbg.predict(valid_x_1)
#print(classification_report(y_valid_1,xgb_pred))

metrics_ML(XGBClassifier(n_estimators=1000, subsample=0.8),  train_x_1, y_train_1, valid_x_1, y_valid_1, LBL, gb=True)

Loading the transformers 2


In [ ]:
model_2 = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')
embeddings_2 = model_2.encode(np.array(df["text"]))
train_x_2, valid_x_2, y_train_2, y_valid_2 = model_selection.train_test_split(np.array(embeddings_2), df["label"], random_state=42, stratify=df["label"], test_size=0.2)


In [ ]:
y_train_2 = y_train_2.astype('int')
y_valid_2 = y_valid_2.astype('int')

In [ ]:
# SVM
svm_clf = SVC(kernel="poly",C=100,gamma=1)
svm_clf.fit(train_x_2,y_train_2)
svm_pred = svm_clf.predict(valid_x_2)
print(classification_report(y_valid_2,svm_pred))

metrics_ML(SVC(kernel="poly",C=100,gamma=1),  train_x_2, y_train_2, valid_x_2, y_valid_2, LBL, gb=True)

In [ ]:
# Random Forest
rf = ensemble.RandomForestClassifier()
rf.fit(train_x_2,y_train_2)
rf_pred = rf.predict(valid_x_2)
#print(classification_report(y_valid_2,rf_pred))

metrics_ML(ensemble.RandomForestClassifier(),  train_x_2, y_train_2, valid_x_2, y_valid_2, LBL, gb=True)

In [ ]:
# Logestic Regression
lr = linear_model.LogisticRegression(max_iter=1000,  random_state=42)
lr.fit(train_x_2,y_train_2)
lr_pred = lr.predict(valid_x_2)
# print(classification_report(y_valid_2,rf_pred))

metrics_ML(linear_model.LogisticRegression(max_iter=1000,  random_state=42),  train_x_2, y_train_2, valid_x_2, y_valid_2, LBL, gb=True)

In [ ]:
# SGD
print("\nStochastic Gradient Descent with early stopping for One-Hot encoding\n")
print("Early Stopping : 10 iterations without change")
sgd = SGDClassifier(loss='modified_huber', max_iter=1000, tol=1e-3,   n_iter_no_change=10, early_stopping=True, n_jobs=-1 )
sgd.fit(train_x_2,y_train_2)
sgd_pred = sgd.predict(valid_x_2)
#print(classification_report(y_valid_2,sgd_pred))

metrics_ML(SGDClassifier(loss='modified_huber', max_iter=1000, tol=1e-3,   n_iter_no_change=10, early_stopping=True, n_jobs=-1 ),  train_x_2, y_train_2, valid_x_2, y_valid_2, LBL, gb=True)

In [ ]:
# XgBOOST
xbg = XGBClassifier(n_estimators=1000, subsample=0.8)
eval_set = [(train_x_2, y_train_2), (valid_x_2, y_valid_2)]
xbg.fit(train_x_2, y_train_2, eval_metric=["error", "logloss"],  eval_set=eval_set,early_stopping_rounds=10, verbose=False) 
xgb_pred = xbg.predict(valid_x_2)
#print(classification_report(y_valid_2,xgb_pred))

metrics_ML(XGBClassifier(n_estimators=1000, subsample=0.8),  train_x_2, y_train_2, valid_x_2, y_valid_2, LBL, gb=True)

Lodading transformer 3 part 1


In [ ]:
le = LabelEncoder()

In [ ]:
def categorical_vals(df_col):
    A=[]
    for i in df_col:
        t = [0]*max(np.unique(df_col)+1)
        t[i]=1
        A.append(t)
    return A

In [ ]:
#model_31 = SentenceTransformer('monsoon-nlp/tamillion')
#embeddings_31 = model_31.encode(np.array(df["text"]))
train_x_31, valid_x_31, y_train_31, y_valid_31 = model_selection.train_test_split(df, df["label"], random_state=42, stratify=df["label"], test_size=0.2)

In [ ]:
eval_frame = pd.DataFrame(train_x_31) 
eval_frame['class'] = categorical_vals(le.fit_transform(df["label"]))
#model_31.fit(train_x_31,y_train_31)
print(eval_frame.head())

In [ ]:
from simpletransformers.classification import MultiLabelClassificationModel
# set use_cuda=False on CPU-only platforms
model = MultiLabelClassificationModel('bert', 'monsoon-nlp/tamillion', num_labels=2, use_cuda=False,
                            args={
                'reprocess_input_data': True,
                'use_cached_eval_features': False,
                'overwrite_output_dir': True,
                'num_train_epochs': 1,
     
})
model.train_model(df)#,eval_data=eval_frame)
print("trained model")

Loading transformer 3

In [ ]:
model_3 = SentenceTransformer('monsoon-nlp/tamillion')
embeddings_3 = model_3.encode(np.array(df["text"]))
train_x_3, valid_x_3, y_train_3, y_valid_3 = model_selection.train_test_split(np.array(embeddings_3), df["label"], random_state=42, stratify=df["label"], test_size=0.2)

In [ ]:
y_train_3 = y_train_3.astype('int')
y_valid_3 = y_valid_3.astype('int')

In [ ]:
# SVM
svm_clf = SVC(kernel="poly",C=100,gamma=1)
svm_clf.fit(train_x_3,y_train_3)
svm_pred = svm_clf.predict(valid_x_3)
#print(classification_report(y_valid_3,svm_pred))

metrics_ML(SVC(kernel="poly",C=100,gamma=1),  train_x_3, y_train_3, valid_x_3, y_valid_3, LBL, gb=True)

In [ ]:
# Random Forest
rf = ensemble.RandomForestClassifier()
rf.fit(train_x_3,y_train_3)
rf_pred = rf.predict(valid_x_3)
#print(classification_report(y_valid_3,rf_pred))

metrics_ML(ensemble.RandomForestClassifier(),  train_x_3, y_train_3, valid_x_3, y_valid_3, LBL, gb=True)

In [ ]:
# Logestic Regression
lr = linear_model.LogisticRegression(max_iter=1000,  random_state=42)
lr.fit(train_x_3,y_train_3)
lr_pred = lr.predict(valid_x_3)
# print(classification_report(y_valid_3,rf_pred))

metrics_ML(linear_model.LogisticRegression(max_iter=1000,  random_state=42),  train_x_3, y_train_3, valid_x_3, y_valid_3, LBL, gb=True)

In [ ]:
# SGD
print("\nStochastic Gradient Descent with early stopping for One-Hot encoding\n")
print("Early Stopping : 10 iterations without change")
sgd = SGDClassifier(loss='modified_huber', max_iter=1000, tol=1e-3,   n_iter_no_change=10, early_stopping=True, n_jobs=-1 )
sgd.fit(train_x_3,y_train_3)
sgd_pred = sgd.predict(valid_x_3)
# print(classification_report(y_valid_3,sgd_pred))

metrics_ML(SGDClassifier(loss='modified_huber', max_iter=1000, tol=1e-3,   n_iter_no_change=10, early_stopping=True, n_jobs=-1 ),  train_x_3, y_train_3, valid_x_3, y_valid_3, LBL, gb=True)

In [ ]:
# XgBOOST
xbg = XGBClassifier(n_estimators=1000, subsample=0.8)
eval_set = [(train_x_3, y_train_3), (valid_x_3, y_valid_3)]
xbg.fit(train_x_3, y_train_3, eval_metric=["error", "logloss"],  eval_set=eval_set,early_stopping_rounds=10, verbose=False) 
xgb_pred = xbg.predict(valid_x_3)
#print(classification_report(y_valid_3,xgb_pred))

metrics_ML(XGBClassifier(n_estimators=1000, subsample=0.8),  train_x_3, y_train_3, valid_x_3, y_valid_3, LBL, gb=True)

Loading transformer 5


In [ ]:
model_5 = SentenceTransformer('google/muril-base-cased')
embeddings_5 = model_5.encode(np.array(df["text"]))
train_x_5, valid_x_5, y_train_5, y_valid_5 = model_selection.train_test_split(np.array(embeddings_5), df["label"], random_state=42, stratify=df["label"], test_size=0.2)

In [ ]:
y_train_5 = y_train_5.astype('int')
y_valid_5 = y_valid_5.astype('int')

In [ ]:
# SVM
svm_clf = SVC(kernel="poly",C=100,gamma=1)
svm_clf.fit(train_x_5,y_train_5)
svm_pred = svm_clf.predict(valid_x_5)
#print(classification_report(y_valid_5,svm_pred))

metrics_ML(SVC(kernel="poly",C=100,gamma=1),  train_x_5, y_train_5, valid_x_5, y_valid_5, LBL, gb=True)

In [ ]:
# Random Forest
rf = ensemble.RandomForestClassifier()
rf.fit(train_x_5,y_train_5)
rf_pred = rf.predict(valid_x_5)
#print(classification_report(y_valid_5,rf_pred))

metrics_ML(ensemble.RandomForestClassifier(),  train_x_5, y_train_5, valid_x_5, y_valid_5, LBL, gb=True)

In [ ]:
# Logestic Regression
lr = linear_model.LogisticRegression(max_iter=1000,  random_state=42)
lr.fit(train_x_5,y_train_5)
lr_pred = lr.predict(valid_x_5)
# print(classification_report(y_valid_5,rf_pred))

metrics_ML(linear_model.LogisticRegression(max_iter=1000,  random_state=42),  train_x_5, y_train_5, valid_x_5, y_valid_5, LBL, gb=True)

In [ ]:
# SGD
print("\nStochastic Gradient Descent with early stopping for One-Hot encoding\n")
print("Early Stopping : 10 iterations without change")
sgd = SGDClassifier(loss='modified_huber', max_iter=1000, tol=1e-3,   n_iter_no_change=10, early_stopping=True, n_jobs=-1 )
sgd.fit(train_x_5,y_train_5)
sgd_pred = sgd.predict(valid_x_5)
# print(classification_report(y_valid_5,sgd_pred))

metrics_ML(SGDClassifier(loss='modified_huber', max_iter=1000, tol=1e-3,   n_iter_no_change=10, early_stopping=True, n_jobs=-1 ),  train_x_5, y_train_5, valid_x_5, y_valid_5, LBL, gb=True)

In [ ]:
# XgBOOST
xbg = XGBClassifier(n_estimators=1000, subsample=0.8)
eval_set = [(train_x_5, y_train_5), (valid_x_5, y_valid_5)]
xbg.fit(train_x_5, y_train_5, eval_metric=["error", "logloss"],  eval_set=eval_set,early_stopping_rounds=10, verbose=False) 
xgb_pred = xbg.predict(valid_x_5)
#print(classification_report(y_valid_5,xgb_pred))

metrics_ML(XGBClassifier(n_estimators=1000, subsample=0.8),  train_x_5, y_train_5, valid_x_5, y_valid_5, LBL, gb=True)

In [ ]:
df.head()
